In [ ]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 96
h = 128
input_size = w * h

In [ ]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img.reshape([1,-1]),label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k:k+1,:],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label
def get_test_batch(batch_size):
    j = np.random.randint(20788,22424,batch_size)
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k:k+1,:],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [ ]:
## num_steps = 500000
batch_size = 64

# Network Parameters
image_dim = input_size # MNIST images are 28x28 pixels
hidden_dim = 4000
latent_dim = 2000
# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [ ]:
# Variables
weights = {
    'encoder_h1': tf.Variable(glorot_init([image_dim, hidden_dim])),
    'z_mean': tf.Variable(glorot_init([hidden_dim, latent_dim])),
    'z_std': tf.Variable(glorot_init([hidden_dim, latent_dim])),
    'decoder_h1': tf.Variable(glorot_init([latent_dim, hidden_dim])),
    'decoder_out': tf.Variable(glorot_init([hidden_dim, image_dim]))
}
biases = {
    'encoder_b1': tf.Variable(glorot_init([hidden_dim])),
    'z_mean': tf.Variable(glorot_init([latent_dim])),
    'z_std': tf.Variable(glorot_init([latent_dim])),
    'decoder_b1': tf.Variable(glorot_init([hidden_dim])),
    'decoder_out': tf.Variable(glorot_init([image_dim]))
}

In [ ]:
# Building the encoder
input_image = tf.placeholder(tf.float32, shape=[None, image_dim])
Y = tf.placeholder(tf.float32,shape=[None,10])
encoder = tf.layers.dense(inputs=input_image,units=hidden_dim,activation=tf.nn.tanh)
z_mean = tf.layers.dense(inputs=encoder,units=latent_dim)
z_std =  tf.layers.dense(inputs=encoder,units=latent_dim)

# Sampler: Normal (gaussian) random distribution
eps = tf.random_normal(tf.shape(z_std), dtype=tf.float32, mean=0., stddev=1.0,
                       name='epsilon')
z = z_mean + tf.exp(z_std / 2) * eps

y_1 = tf.layers.dense(inputs=z_mean,units=500,name='alp',activation=tf.nn.sigmoid,kernel_initializer=tf.variance_scaling_initializer)
y_11 = tf.layers.dense(inputs=y_1,units=100,name='alp2',activation=tf.nn.sigmoid,kernel_initializer=tf.variance_scaling_initializer)
y_2 = tf.layers.dense(inputs=y_11,units=10,name='alp3')
clf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_2))
alper_list = [v for v in tf.trainable_variables() if 'alp' in v.name]
lrr = tf.placeholder(shape=[],dtype=tf.float32)
optimizer2 = tf.train.RMSPropOptimizer(lrr).minimize(clf_loss,var_list=alper_list)
correctPred = tf.equal(tf.argmax(y_2,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


# Building the decoder (with scope to re-use these layers later)
decoder_h1 = tf.layers.dense(inputs=z,units=hidden_dim,activation=tf.nn.tanh)
decoder = tf.layers.dense(inputs=decoder_h1,units=input_size)


In [ ]:
# Define VAE Loss
def vae_loss(x_reconstructed, x_true):
    # Reconstruction loss
    #encode_decode_loss = x_true * tf.log(1e-5 + x_reconstructed) \
    #                     + (1 - x_true) * tf.log(1e-5 + 1 - x_reconstructed)
    #encode_decode_loss = -tf.reduce_sum(encode_decode_loss, 1)
    encode_decode_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=x_true,logits=x_reconstructed))
    # KL Divergence loss
    kl_div_loss = 1 + z_std - tf.square(z_mean) - tf.exp(z_std)
    kl_div_loss = -0.5 * tf.reduce_sum(kl_div_loss, 1)
    print(encode_decode_loss,kl_div_loss)
    return tf.reduce_mean(encode_decode_loss  + kl_div_loss * 0.0005)

print(decoder.shape)
print(input_image.shape)
loss_op = vae_loss(decoder, input_image)
optimizer = tf.train.RMSPropOptimizer(lrr)
train_op = optimizer.minimize(loss_op)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
loss_array = []
clf_array = []
learn =0.0001

In [ ]:
i=1
while(1): 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    if 1:
        batch_x,_ = get_batch(128)
        feed_dict = {input_image: batch_x,lrr:learn*0.001}
        _, l = sess.run([train_op, loss_op], feed_dict=feed_dict)
        loss_array.append(l)
        print('Recons:',l)
    if 0 :
        batch_x,batch_y = get_batch(128)
        feed_dict = {input_image: batch_x,Y:batch_y,lrr:learn}
        _, l = sess.run([optimizer2, clf_loss], feed_dict=feed_dict)
        clf_array.append(l)
        print('Class:',l)
    if i % 10 == 0:
        #show(batch_x[0:1,:])
        print('Step %i, Loss: %f' % (i, l))
        show(batch_x[0:1,:])
        img = sess.run(decoder,feed_dict=feed_dict)
        show(img[0:1,:])
        learn *= 1
    i+=1

In [ ]:
total_acc = 0
for i in range(100):
        batch_x,label = get_test_batch(batch_size)
        feed_dict = {input_image: batch_x,Y:label}
        l,acc = sess.run([clf_loss,accuracy], feed_dict=feed_dict)
        total_acc += acc
print(total_acc/100)

In [ ]:
plt.semilogy(loss_array,label='Recons')
#plt.semilogy(clf_array,label='Class')
plt.legend()
plt.show()

In [ ]:
learn = 0.001

In [ ]:
learn